In [17]:
import pandas as pd
import numpy as np
PATHS = {
    "gasto": "~/telecom-iag/data/S1_05_GASTO_TELECOMUNICACIONES.csv",
    "demografia": "~/telecom-iag/data/S1_01_INF_DEMOGRAFICA.csv",
    "infraestructura": "~/telecom-iag/data/S1_03_SERVICIOS.csv",
    "adopcion": "~/telecom-iag/data/S2_BAF_POR_TECONOLOGIA.csv",
    "disponibilidad": "~/telecom-iag/data/S1_06_DISPONIBILIDAD_TIC.csv"
}

In [18]:
def process_gasto(path):
    """
    Procesa el gasto
    Agrupa por entidad, calcula el gasto promedio de fijos y móviles, 
    y los suma en GASTO_TOTAL_PROMEDIO.
    """
    try:
        df = pd.read_csv(path)
        df = df[~df['ENTIDAD'].str.contains('Nacional', case=False, na=False)].copy()
        
        # Agrupar por entidad y calcular el promedio de gasto (a través de deciles)
        df_gasto = df.groupby('ENTIDAD')[['GASTO_PM_FIJAS', 'GASTO_PM_MOVILES']].mean()
        # Crear la variable dependiente final
        df_gasto['GASTO_TOTAL_PROMEDIO'] = df_gasto['GASTO_PM_FIJAS'] + df_gasto['GASTO_PM_MOVILES']
        return df_gasto
    except FileNotFoundError:
        print(f"\n--- ERROR CRÍTICO (Archivo No Encontrado) ---")
        print(f"  Función: process_gasto")
        print(f"  Ruta esperada: {path}")
        #
        return None
    except Exception as e:
        print(f"\n--- ERROR CRÍTICO (Error de Procesamiento) ---")
        print(f"  Función: process_gasto")
        print(f"  Archivo: {path}")
        print(f"  Error: {e}")
        #
        return None

def process_demografia(path):
    """
    Procesa los datos demográficos Filtra por total de entidad (ZONA == 'Nacional') y elimina el total del país.
    Limpia las columnas numéricas.
    """
    try:
        df = pd.read_csv(path)
        # Filtrar por el total de la entidad (ZONA == 'Nacional')
        df_dem = df[df['ZONA'] == 'Nacional'].copy()
        
        #
        # Eliminar el agregado del país (ENT != 'Nacional')
        # Filtro robusto: elimina cualquier fila donde ENT contenga 'Nacional'
        df_dem = df_dem[~df_dem['ENT'].str.contains('Nacional', case=False, na=False)].copy()

        # Limpiar números (quitar comas)
        df_dem['HOGARES'] = pd.to_numeric(df_dem['HOGARES'].replace(',', '', regex=True))
        df_dem['HABITANTES'] = pd.to_numeric(df_dem['HABITANTES'].replace(',', '', regex=True))
        # Renombrar columna clave para unir
        df_dem.rename(columns={'ENT': 'ENTIDAD'}, inplace=True)
        return df_dem.set_index('ENTIDAD')[['HOGARES', 'HABITANTES']]
    except FileNotFoundError:
        print(f"\n--- ERROR CRÍTICO (Archivo No Encontrado) ---")
        print(f"  Función: process_demografia")
        print(f"  Ruta esperada: {path}")
        #
        return None
    except Exception as e:
        print(f"\n--- ERROR CRÍTICO (Error de Procesamiento) ---")
        print(f"  Función: process_demografia")
        print(f"  Archivo: {path}")
        print(f"  Error: {e}")
        #
        return None

def process_infraestructura(path):
    """
    Procesa los datos de infraestructura 
    Pivota la tabla para tener servicios como columnas.
    """
    try:
        df = pd.read_csv(path)
        
        #
        # Eliminar cualquier fila que contenga 'Nacional' en ENTIDAD antes de procesar
        df = df[~df['ENTIDAD'].str.contains('Nacional', case=False, na=False)].copy()
        
        # Pivotar la tabla
        df_infra = df.pivot(index='ENTIDAD', columns='SERVICIO', values='LINEAS_ACCESOS')
        return df_infra
    except FileNotFoundError:
        print(f"\n--- ERROR CRÍTICO (Archivo No Encontrado) ---")
        print(f"  Función: process_infraestructura")
        print(f"  Ruta esperada: {path}")
        #
        return None
    except Exception as e:
        print(f"\n--- ERROR CRÍTICO (Error de Procesamiento) ---")
        print(f"  Función: process_infraestructura")
        print(f"  Archivo: {path}")
        print(f"  Error: {e}")
        #
        return None

def process_adopcion(path):
    """
    Procesa la adopción de tecnología
    Filtra por el período más reciente y pivota la tabla.
    """
    try:
        df = pd.read_csv(path)

        #
        # Eliminar cualquier fila que contenga 'Nacional' en ENTIDAD antes de procesar
        df = df[~df['ENTIDAD'].str.contains('Nacional', case=False, na=False)].copy()
        
        # --- ¡NUEVO FILTRO! ---
        # Eliminar la fila 'Total' de las tecnologías para evitar la columna de ceros
        df = df[~df['TECNOLOGIA'].str.contains('Total', case=False, na=False)].copy()

        # Encontrar el período más reciente
        periodo_reciente = df['PERIODO'].max()
        print(f"Filtrando adopción por período más reciente: {periodo_reciente}")
        df_reciente = df[df['PERIODO'] == periodo_reciente].copy()
        # Pivotar
        df_adopcion = df_reciente.pivot(index='ENTIDAD', columns='TECNOLOGIA', values='PORCENTAJE')
        # Llenar NaNs con 0 (asumiendo que si no hay reporte es 0%)
        return df_adopcion.fillna(0)
    except FileNotFoundError:
        print(f"\n--- ERROR CRÍTICO (Archivo No Encontrado) ---")
        print(f"  Función: process_adopcion")
        print(f"  Ruta esperada: {path}")
        return None
    except Exception as e:
        print(f"\n--- ERROR CRÍTICO (Error de Procesamiento) ---")
        print(f"  Función: process_adopcion")
        print(f"  Archivo: {path}")
        print(f"  Error: {e}")
        return None

def process_disponibilidad(path):
    """
    Procesa la disponibilidad de TICs (Sección 3.5).
    Filtra por total de entidad y pivota.
    """
    try:
        df = pd.read_csv(path)
        # Filtrar por el total de la entidad (ZONA == 'Nacional')
        df_disp = df[df['ZONA'] == 'Nacional'].copy()
        
        #
        # Eliminar el agregado del país (ENT != 'Nacional')
        # Filtro robusto: elimina cualquier fila donde ENT contenga 'Nacional'
        df_disp = df_disp[~df_disp['ENT'].str.contains('Nacional', case=False, na=False)].copy()

        # Pivotar
        df_disp_pivot = df_disp.pivot(index='ENT', columns='EQUIPO', values='PORCENTAJE')
        # Renombrar índice para unir
        df_disp_pivot.index.name = 'ENTIDAD'
        return df_disp_pivot
    except FileNotFoundError:
        print(f"\n--- ERROR CRÍTICO (Archivo No Encontrado) ---")
        print(f"  Función: process_disponibilidad")
        print(f"  Ruta esperada: {path}")
        #
        return None
    except Exception as e:
        print(f"\n--- ERROR CRÍTICO (Error de Procesamiento) ---")
        print(f"  Función: process_disponibilidad")
        print(f"  Archivo: {path}")
        print(f"  Error: {e}")
        #
        return None

def clean_column_names(df):
    """Limpia los nombres de las columnas para statsmodels."""
    df.columns = df.columns.str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.replace('ó', 'o', regex=True)
    df.columns = df.columns.str.replace('í', 'i', regex=True)
    return df

In [19]:
df_gasto = process_gasto(PATHS['gasto'])
df_demografia = process_demografia(PATHS['demografia'])
df_infraestructura = process_infraestructura(PATHS['infraestructura'])
df_adopcion = process_adopcion(PATHS['adopcion'])
df_disponibilidad = process_disponibilidad(PATHS['disponibilidad'])
dfs_to_merge = [
    df_gasto, 
    df_demografia, 
    df_infraestructura, 
    df_adopcion, 
    df_disponibilidad
]

Filtrando adopción por período más reciente: 2023-I


In [20]:
# Verificar que todos los DFs tengan contenido
if any(df is None for df in dfs_to_merge):
    print("\n--- ERROR: Fallo en la carga de datos ---")
else:
    # Unir todos los DataFrames usando el índice (ENTIDAD)
    df_master = pd.concat(dfs_to_merge, axis=1)

    # Limpiar nombres de columnas para el modelo
    df_master = clean_column_names(df_master)

    # Eliminar filas con valores NaN (si alguna entidad no está en todos los archivos)
    df_master_clean = df_master.dropna()

    print("\n--- Base de Datos Maestra (df_master_clean) Creada ---")
    print(df_master_clean.head())
    print("\nColumnas disponibles:", df_master_clean.columns.tolist())
    df_master_clean.to_csv("~/telecom-iag/data/master_data.csv", index=False)
    


--- Base de Datos Maestra (df_master_clean) Creada ---
                     GASTO_PM_FIJAS  GASTO_PM_MOVILES  GASTO_TOTAL_PROMEDIO  \
ENTIDAD                                                                       
Aguascalientes              496.530           390.533               887.063   
Baja California Sur         501.124           462.204               963.328   
Campeche                    420.226           333.787               754.013   
Chiapas                     317.121           242.936               560.057   
Chihuahua                   501.512           405.209               906.721   

                       HOGARES  HABITANTES  Internet_fijo  Internet_movil  \
ENTIDAD                                                                     
Aguascalientes        413318.0   1481758.0           73.0            91.0   
Baja California Sur   277115.0    839831.0           80.0           103.0   
Campeche              265926.0    949288.0           51.0            95.0   
Chiap

In [21]:
df_master_clean.describe()

,GASTO_PM_FIJAS,GASTO_PM_MOVILES,GASTO_TOTAL_PROMEDIO,HOGARES,HABITANTES,Internet_fijo,Internet_movil,TV_Restringida,Telefonia_fija,Telefonia_movil,Cable_Coaxial,DSL,Fibra_optica,Sin_tecnologia_especificada,Tecnologia_movil
count,28.000000,28.000000,28.000000,2.800000e+01,2.800000e+01,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,455.984429,364.434071,820.418500,1.123956e+06,3.874803e+06,62.642857,92.821429,63.714286,61.214286,105.535714,20.513214,14.949643,43.613571,18.255714,2.668214
std,58.060667,64.114722,119.413229,1.015156e+06,3.468158e+06,18.276101,14.244046,15.109127,26.830548,10.598680,13.438792,6.865805,13.275600,17.000680,1.096750
min,317.121000,240.199000,560.057000,2.488880e+05,7.698510e+05,27.000000,52.000000,38.000000,22.000000,79.000000,0.000000,4.120000,17.300000,0.000000,0.320000
25%,420.265750,326.617000,747.005750,5.072020e+05,1.810074e+06,50.500000,86.000000,54.000000,46.750000,98.750000,10.505000,10.587500,35.402500,3.570000,1.882500
50%,463.244000,354.589000,820.274500,8.587940e+05,2.930796e+06,61.000000,95.000000,60.500000,55.000000,109.000000,24.725000,13.650000,44.705000,9.215000,2.510000
75%,491.742250,413.483500,902.724000,1.286982e+06,4.605076e+06,75.500000,103.000000,73.250000,69.750000,113.000000,29.530000,17.977500,52.360000,32.715000,3.590000
max,586.036000,497.409000,1083.445000,4.969467e+06,1.729898e+07,100.000000,116.000000,106.000000,156.000000,121.000000,48.370000,35.180000,79.190000,55.880000,4.670000
